Created on 18-12-2025

@author: Xavier Mousset

In [2]:
import os
import sys
import math
import webbrowser
from pathlib import Path
from abc import abstractmethod
from typing import Literal, List, Any

from IPython.display import clear_output

import sqlite3
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import qualitative, sequential

lmt_analysis_path = Path.cwd().parent
sys.path.append(lmt_analysis_path.as_posix())

from dim_c_brains.scripts.reports_manager import HTMLReportManager
from dim_c_brains.scripts.events import generic_events_list
from dim_c_brains.scripts.data_extractor import DataFrameCreator, LargeDataFrameCreator
from dim_c_brains.scripts.plotting import plt_curve_shaded

from lmtanalysis.Animal import Animal, AnimalPool
from lmtanalysis.Measure import oneDay, oneHour, oneMinute
from lmtanalysis.Event import EventTimeLine
from lmtanalysis.ParametersMouse import ParametersMouse

COLORS = {
    "discrete": qualitative.Bold[::-1],
    "continuous": sequential.Plotly3
}

# data_path = Path.cwd() / "res" / "data" / "exemple data - 20180110_validation_4_ind_Experiment_6644_e.sqlite"
data_path = Path.home() / "Syncnot" / "lmt-blocks" / "experiments" / "xmd" / "nadege" / "groupe1-cage1-LMT1.sqlite"

In [3]:
connection = sqlite3.connect(data_path.as_posix())

# reports_manager = HTMLReportManager(exp_name="example_analysis")
start_time = 12*oneHour
end_time = 13*oneHour

clear_output()

# print(f"Speeds memory usage: {sys.getsizeof(animal_pool.animalDictionary[1].speeds)/10**6:.2f} Mo")

# Test

In [4]:
data = LargeDataFrameCreator(
    connection= connection,
    chunk_size= oneDay
)

df_event = data.get_largedf_events(events= "Oral-oral Contact")
# df_move = data.get_largedf_movement()

clear_output()

In [5]:
df_event.head()

,RFID,ANIMALID,EVENT,FRAMENUMBER,EVENT_COUNT,FRAME_COUNT,TIME
0,002035269594,1,Oral-oral Contact,15712,55,303,2025-12-02 10:30:00
1,002035269594,1,Oral-oral Contact,42712,115,504,2025-12-02 10:45:00
2,002035269594,1,Oral-oral Contact,69712,105,640,2025-12-02 11:00:00
3,002035269594,1,Oral-oral Contact,96712,99,400,2025-12-02 11:15:00
4,002035269594,1,Oral-oral Contact,123712,78,408,2025-12-02 11:30:00


In [6]:
x_col = "TIME"
y_col = "SPEED_MEAN"
y_col_std = "SPEED_STD"
color = "RFID"

plt_curve_shaded(
    df_move, "TIME", "SPEED_MEAN",
    y_min_col= "SPEED_MIN",
    y_max_col= "SPEED_MAX",
    color= "RFID",
    color_discrete_sequence=COLORS["discrete"]
)

NameError: name 'df_move' is not defined

In [7]:
px.bar(
    df_event,
    x="TIME",
    y="EVENT_COUNT",
    color="RFID",
    title="Oral-oral Contact events per animal over time",
    color_discrete_sequence=COLORS["discrete"]
).show()

In [8]:
px.bar(
    df_event,
    x="TIME",
    y="FRAME_COUNT",
    color="RFID",
    title="Oral-oral Contact events per animal over time",
    color_discrete_sequence=COLORS["discrete"]
).show()

In [9]:
if df_event is None:
    raise ValueError("No event data available.")

df_event["HOUR"] = df_event["TIME"].apply(lambda x: x.hour)
df_event.head(6)

,RFID,ANIMALID,EVENT,FRAMENUMBER,EVENT_COUNT,FRAME_COUNT,TIME,HOUR
0,002035269594,1,Oral-oral Contact,15712,55,303,2025-12-02 10:30:00,10
1,002035269594,1,Oral-oral Contact,42712,115,504,2025-12-02 10:45:00,10
2,002035269594,1,Oral-oral Contact,69712,105,640,2025-12-02 11:00:00,11
3,002035269594,1,Oral-oral Contact,96712,99,400,2025-12-02 11:15:00,11
4,002035269594,1,Oral-oral Contact,123712,78,408,2025-12-02 11:30:00,11
5,002035269594,1,Oral-oral Contact,150712,76,597,2025-12-02 11:45:00,11


In [10]:
if df_event is None:
    raise ValueError("No event data available.")

# df_plot = (
#     df_event.groupby(["RFID", "HOUR"], observed=True)
#     .size()
#     .reset_index(name="COUNT")
#     .sort_values(by="HOUR")
# )

df_plot = (
    df_event.groupby(["RFID", "HOUR"], observed=True)["EVENT_COUNT"]
    .sum()
    .reset_index(name="COUNT")
    .sort_values(by="HOUR")
)

df_plot["HOUR"] = df_plot["HOUR"].astype(str) + "h"

df_plot.head(6)

,RFID,HOUR,COUNT
0,002035269579,0h,567
24,002035269583,0h,896
48,002035269592,0h,634
72,002035269594,0h,268
96,002035269625,0h,558
1,002035269579,1h,118


In [11]:
fig = px.bar_polar(
    df_plot, r="COUNT", theta="HOUR",
    color="RFID", color_discrete_sequence= COLORS["discrete"]
)

fig = px.line_polar(
    df_plot, r="COUNT", theta="HOUR", line_close= True,
    color="RFID", color_discrete_sequence= COLORS["discrete"]
)

# fig.update_layout(
#     polar=dict(
#         radialaxis=dict(title=dict(text="Trial count"))
#     ),
#     legend_title="RFID"
# )

fig.show()

In [12]:
df_plot = (
    df_event.groupby(["RFID"], observed=True)["EVENT_COUNT"]
    .sum()
    .reset_index(name="COUNT")
)

px.bar(
    df_plot,
    x="RFID",
    y="COUNT",
    color="RFID",
    title="Oral-oral Contact events per animal over time",
    color_discrete_sequence=COLORS["discrete"]
).show()

In [20]:
fig = px.histogram(df_event, x="FRAME_COUNT", color= "RFID", nbins=60, title="Frame Count Histogram",
                   histogram)
mean = df_event["FRAME_COUNT"].mean()
std = df_event["FRAME_COUNT"].std()
fig.add_vline(x=mean, line_dash="dash", line_color="red", annotation_text="Mean")
fig.add_vrect(x0=mean-std, x1=mean+std, fillcolor="red", opacity=0.2, line_width=0)
fig.show()

In [21]:
fig = px.violin(df_event, y="FRAME_COUNT", color= "RFID", points="all", title="Frame Count Violin Plot")
fig.show()

In [ ]:
df_plot.head()

,RFID,COUNT
0,002035269579,8588
1,002035269583,9742
2,002035269592,8947
3,002035269594,7677
4,002035269625,9905


# Distance vs Time